## Building Predictive Models

In [14]:
import pandas as pd
import os
import numpy as np

### Import Data

In [4]:
# Set paths to processed data
processed_data_path = os.path.join(os.path.pardir, 'data', 'processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
test_file_path = os.path.join(processed_data_path, 'test.csv')

In [5]:
train_df = pd.read_csv(train_file_path, index_col="PassengerId")
test_df = pd.read_csv(test_file_path, index_col="PassengerId")

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Survived            891 non-null    int64  
 1   Age                 891 non-null    float64
 2   Fare                891 non-null    float64
 3   FamilySize          891 non-null    int64  
 4   IsMother            891 non-null    int64  
 5   IsMale              891 non-null    int64  
 6   Deck_A              891 non-null    int64  
 7   Deck_B              891 non-null    int64  
 8   Deck_C              891 non-null    int64  
 9   Deck_D              891 non-null    int64  
 10  Deck_E              891 non-null    int64  
 11  Deck_F              891 non-null    int64  
 12  Deck_G              891 non-null    int64  
 13  Deck_Z              891 non-null    int64  
 14  Pclass_1            891 non-null    int64  
 15  Pclass_2            891 non-null    int64  
 16  Pclass_3

In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 418 non-null    float64
 1   Fare                418 non-null    float64
 2   FamilySize          418 non-null    int64  
 3   IsMother            418 non-null    int64  
 4   IsMale              418 non-null    int64  
 5   Deck_A              418 non-null    int64  
 6   Deck_B              418 non-null    int64  
 7   Deck_C              418 non-null    int64  
 8   Deck_D              418 non-null    int64  
 9   Deck_E              418 non-null    int64  
 10  Deck_F              418 non-null    int64  
 11  Deck_G              418 non-null    int64  
 12  Deck_Z              418 non-null    int64  
 13  Pclass_1            418 non-null    int64  
 14  Pclass_2            418 non-null    int64  
 15  Pclass_3            418 non-null    int64  
 16  Title

### Data Preparation

In [9]:
X = train_df.loc[:, 'Age':].to_numpy().astype('float') # all rows and all columns. Convert to matrix and float
y = train_df['Survived'].ravel() # flattened one dimensional cell

In [10]:
print(X.shape, y.shape) #upper case for matrix X, lower case for one-dimensional y

(891, 32) (891,)


In [15]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(712, 32) (712,)
(179, 32) (179,)


In [17]:
# average survival in train and test
# You want to have positive to be evanly distributed among test and train data
print('mean survival in train: {0:.3f}'.format(np.mean(y_train)))
print('mean survival in test : {0:.3f}'.format(np.mean(y_test)))

mean survival in train: 0.383
mean survival in test : 0.385


### Check Scikit-Learn Version

In [18]:
import sklearn

In [19]:
sklearn.__version__

'0.24.2'

## Baseline Model

In [20]:
# Import function
from sklearn.dummy import DummyClassifier

In [21]:
# create model
model_dummy = DummyClassifier(strategy='most_frequent', random_state=0)

In [22]:
# train model. Provide input data and output data
model_dummy.fit(X_train, y_train)

DummyClassifier(random_state=0, strategy='most_frequent')

In [24]:
print('score for baseline model : {0:.2f}'.format(model_dummy.score(X_test, y_test))) # Accuracy

score for baseline model : 0.61


In [25]:
# performance metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [26]:
# accuracy score
print('accuracy of baseline model : {0:.2f}'.format(accuracy_score(y_test, model_dummy.predict(X_test))))

accuracy of baseline model : 0.61


In [27]:
# Confusion matrix
print('confusion matrix for baseline model : \n {0}'.format(confusion_matrix(y_test, model_dummy.predict(X_test))))

confusion matrix for baseline model : 
 [[110   0]
 [ 69   0]]


In [28]:
# Precision and recall score
print('precision for baseline model : {0:.2f}'.format(precision_score(y_test, model_dummy.predict(X_test))))
print('recall for baseline model : {0:.2f}'.format(recall_score(y_test, model_dummy.predict(X_test))))

precision for baseline model : 0.00
recall for baseline model : 0.00
/Users/marcin.kokott/opt/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## First Kaggle Submission

In [29]:
# converting to the matrix
test_X = test_df.to_numpy().astype('float')

In [30]:
# get predictions
predictions = model_dummy.predict(test_X)

In [31]:
df_submission = pd.DataFrame({'PassengerId': test_df.index, 'Survived' : predictions})

In [32]:
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [33]:
submissions_data_path = os.path.join(os.path.pardir, 'data', 'external')
submissions_file_path = os.path.join(submissions_data_path, '01_dummy.csv')

In [34]:
df_submission.to_csv(submissions_file_path, index=False)

In [35]:
def get_submission_file(model, filename):
    # converting to the matrix
    test_X = test_df.to_numpy().astype('float')
    # get predictions
    predictions = model.predict(test_X)
    # submission data frame
    df_submission = pd.DataFrame({'PassengerId': test_df.index, 'Survived' : predictions})
    # submission file
    submissions_data_path = os.path.join(os.path.pardir, 'data', 'external')
    submissions_file_path = os.path.join(submissions_data_path, filename)
    # write to the file
    df_submission.to_csv(submissions_file_path, index=False)

In [36]:
# get submission file
get_submission_file(model_dummy, '01_dummy.csv')

## Logistic Regression Model

In [37]:
# import function
from sklearn.linear_model import LogisticRegression

In [40]:
# Create model
model_lr_1 = LogisticRegression(max_iter=1000, random_state=0)

In [41]:
# train model
model_lr_1.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=0)

In [43]:
# evalate the model
print('score for logistics regression - version 1 : {0:.2f}'.format(model_lr_1.score(X_test, y_test)))

score for logistics regression - version 1 : 0.83


In [44]:
# accuracy score
print('accuracy of logistic regression - version 1 : {0:.2f}'.format(accuracy_score(y_test, model_lr_1.predict(X_test))))
# Confusion matrix
print('confusion matrix for logistic regression - version 1 : \n {0}'.format(confusion_matrix(y_test, model_lr_1.predict(X_test))))
# Precision and recall score
print('precision for logistic regression - version 1 : {0:.2f}'.format(precision_score(y_test, model_lr_1.predict(X_test))))
print('recall for logistic regression - version 1 : {0:.2f}'.format(recall_score(y_test, model_lr_1.predict(X_test))))

accuracy of logistic regression - version 1 : 0.83
confusion matrix for logistic regression - version 1 : 
 [[95 15]
 [15 54]]
precision for logistic regression - version 1 : 0.78
recall for logistic regression - version 1 : 0.78


In [45]:
# model coefficience
# model weights / model parameters
model_lr_1.coef_

array([[-0.03283038,  0.00421566, -0.52731903,  0.63336717, -1.08177135,
        -0.02275457, -0.29598543, -0.52922391,  0.40221863,  0.97503158,
         0.28566018, -0.28933668, -0.51784081,  0.59204828,  0.13280808,
        -0.71708737,  0.11143556,  1.10707197,  0.19383206, -1.54413377,
         0.78427273, -0.14577265, -0.4989369 , -0.1201452 , -0.03890315,
         0.00670494,  0.16011241,  0.12097377,  0.11907741, -0.23228218,
        -0.18051276,  0.18828175]])

## Second Kaggle Submission

In [46]:
# get submission file
get_submission_file(model_lr_1, '02-lr.csv')

## Part 2

### Hyperparameter Optimization

In [62]:
# base model
model_lr = LogisticRegression(max_iter=10000, random_state=0)

In [63]:
from sklearn.model_selection import GridSearchCV

In [64]:
parameters = {'C':[1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty' : ['l1','l2']}
clf = GridSearchCV(model_lr, param_grid=parameters, cv=3)

In [65]:
clf.fit(X_train, y_train) 

/Users/marcin.kokott/opt/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/marcin.kokott/opt/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/marcin.kokott/opt/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/marcin.kokott/opt/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 

GridSearchCV(cv=3, estimator=LogisticRegression(max_iter=10000, random_state=0),
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']})

In [66]:
clf.best_params_

{'C': 1.0, 'penalty': 'l2'}

In [67]:
print('best score : {0:.2f}'.format(clf.best_score_))

best score : 0.83


In [68]:
# Evaluate model
print('score for logistic regression - version 2 : {0:.2f}'.format(clf.score(X_test, y_test)))

score for logistic regression - version 2 : 0.83


## Making third submission

In [69]:
# get submission file
get_submission_file(clf, '03-lr.csv')

## Feature Normalization and Standardization

In [70]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

### Feature Normalization

In [71]:
# feature normalization
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [72]:
X_train_scaled[:,0].min(), X_train_scaled[:,0].max()

(0.0, 1.0)

### Feature Standardization

In [74]:
# Feature standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [75]:
### Create model after standardization

In [80]:
# base model
model_lr = LogisticRegression()
parameters = {'C':[1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty' : ['none','l2']}
clf = GridSearchCV(model_lr, param_grid=parameters, cv=3)
clf.fit(X_train_scaled, y_train)

/Users/marcin.kokott/opt/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/marcin.kokott/opt/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/marcin.kokott/opt/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/marcin.kokott/opt/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/marcin.kokott/opt/miniconda3/envs/minimal_ds/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting

In [81]:
clf.best_score_

0.8089978607476747

In [82]:
# Evaluate model
print('score for logistic regression - version 2 : {0:.2f}'.format(clf.score(X_test, y_test)))

score for logistic regression - version 2 : 0.72


### Model Persistance

In [91]:
# import pickle library
import pickle

In [92]:
# create the file path
model_file_path = os.path.join(os.path.pardir, 'models', 'lr_model.pkl')
scaler_file_path = os.path.join(os.path.pardir, 'models', 'lr_scaler.pkl')

In [93]:
# open the file to write
model_file_pickle = open(model_file_path, 'wb')
scaler_file_pickle = open(scaler_file_path, 'wb')

In [94]:
# persist the model and scaler
pickle.dump(clf, model_file_pickle)
pickle.dump(scaler, scaler_file_pickle)

In [95]:
# close the file
model_file_pickle.close()
scaler_file_pickle.close()

### load the persisted file

In [100]:
# open files in read mode
model_file_pickle = open(model_file_path, 'rb')
scaler_file_pickle = open(scaler_file_path, 'rb')
# load files
clf_loaded = pickle.load(model_file_pickle)
scaler_loaded = pickle.load(scaler_file_pickle)
# close files
model_file_pickle.close()
scaler_file_pickle.close() 

In [101]:
clf_loaded

GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['none', 'l2']})

In [102]:
scaler_loaded

StandardScaler()

In [103]:
# transform the test data using loaded scaler object
X_test_scaled = scaler_loaded.transform(X_test)
#calculate the score using loaded model object
print('score for persisted logistic regression : {0:.2f}'.format(clf_loaded.score(X_test_scaled, y_test)))

score for persisted logistic regression : 0.84
